In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# CS559 Homework 6

In this computer assignment, we will study *unsupervised learning* using the classical Iris dataset. We will begin with a simple *competitive learning* network and then compare it with the standard $k$-means clustering algorithm.

## 1. Load the Iris dataset.

It contains 150 samples, each with 4 real-valued features, and each sample belongs to one of 3 species. For the purpose of **clustering**, however, we will ignore the labels initially. Let $X$ be the $150 \times 4$ feature matrix, and standardize its columns to have zero mean and unit variance.

In [25]:
iris = pd.read_csv('iris.csv')  # Load the Iris dataset 
iris

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [26]:
# Normalize numeric columns of Iris dataframe
# Keep label in final dataframe
iris_mean = iris.mean(numeric_only=True)
iris_std = iris.std(numeric_only=True)

iris_normalized = (iris.select_dtypes(include=[np.number]) - iris_mean) / iris_std
iris_normalized['variety'] = iris['variety']
iris_normalized.describe()

,sepal.length,sepal.width,petal.length,petal.width
count,1.500000e+02,1.500000e+02,1.500000e+02,1.500000e+02
mean,-5.684342e-16,-7.815970e-16,-2.842171e-16,-3.789561e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.863780e+00,-2.425820e+00,-1.562342e+00,-1.442245e+00
25%,-8.976739e-01,-5.903951e-01,-1.222456e+00,-1.179859e+00
50%,-5.233076e-02,-1.315388e-01,3.353541e-01,1.320673e-01
75%,6.722490e-01,5.567457e-01,7.602115e-01,7.880307e-01
max,2.483699e+00,3.080455e+00,1.779869e+00,1.706379e+00


# 2. We will implement a simple competitive-learning network with 3 neurons.

Let the weight vectors be $w_1,w_2,w_3 \in \mathbb{R}^4$. Initialize them by choosing three random training samples.

Now perform competitive learning for a number of epochs: given an input vector $x$, compute its distances to the three weight vectors, find the winner (the closest weight), and update only the winner via

$$
w_\text{winner} \leftarrow w_\text{winner} + \eta (x - w_\text{winner})
$$

Use a decaying learning rate $\eta$ if needed. At the end of training, record the final weight vectors.

In [27]:
class CompetitiveNet():
    def __init__(self, weights_init):
        self.w = weights_init
        
    def train(self, X, eta, n_epochs):
        for epoch in range(n_epochs):
            for x in X:
                # Compute distances from sample to each weight vector
                distances = np.linalg.norm(self.w - x, axis=1)
                
                # Find index of winning neuron (closest weight vector)
                winner_idx = np.argmin(distances)

                # Update the winning weight vector
                self.w[winner_idx] += eta * (x - self.w[winner_idx])
                
    def cluster_indices(self, X):
        indices = []
        for x in X:
            distances = np.linalg.norm(self.w - x, axis=1)
            winner_idx = np.argmin(distances)
            indices.append(winner_idx)
        return indices
                

# Seed rng
np.random.seed(12345)

# Select 3 random samples from X as initial weights
X = iris_normalized.select_dtypes(include=[np.number]).to_numpy()
initial_weights = X[np.random.choice(X.shape[0], 3, replace=False)]

# Instantitate CompetitiveNet with initial weights                           
nn = CompetitiveNet(initial_weights)

# Train the network
nn.train(X, eta=0.1, n_epochs=100)

# Final weights after training
print("Final weights after training competitive network:")
print(nn.w)

Final weights after training competitive network:
[[ 0.08983553 -0.75613451  0.58914914  0.59362298]
 [-1.0693572   0.76666025 -1.29856269 -1.24099658]
 [ 1.09856075  0.17285005  1.03842785  1.28062897]]


# 3. After training, assign each of the 150 samples to one of the 3 clusters by finding the nearest weight vector.

This produces cluster indices in $\{0, 1, 2\}$ for each sample. Explain in your report why the numerical values 0,1,2 of the cluster indices have *no direct meaning*, and why they cannot be directly compared to the Iris labels $\{0,1,2\}$.

In [28]:
cluster_indices = nn.cluster_indices(X)
iris_normalized['cluster_index_competitive'] = cluster_indices
iris_normalized

,sepal.length,sepal.width,petal.length,petal.width,variety,cluster_index_competitive
0,-0.897674,1.015602,-1.335752,-1.311052,Setosa,1
1,-1.139200,-0.131539,-1.335752,-1.311052,Setosa,1
2,-1.380727,0.327318,-1.392399,-1.311052,Setosa,1
3,-1.501490,0.097889,-1.279104,-1.311052,Setosa,1
4,-1.018437,1.245030,-1.335752,-1.311052,Setosa,1
...,...,...,...,...,...,...
145,1.034539,-0.131539,0.816859,1.443994,Virginica,2
146,0.551486,-1.278680,0.703564,0.919223,Virginica,0
147,0.793012,-0.131539,0.816859,1.050416,Virginica,2
148,0.430722,0.786174,0.930154,1.443994,Virginica,2


The cluster indices from the competitive network are just the indices of the weights associated with each neuron. Those indices are arbitrary; we don't know *which* of the 3 iris species they refer to, only that they indicate three distinct groups. The labels in the Iris dataset correspond to specific iris species. This is also a somewhat arbitrary labeling; the point is that *our* labeling is not necessarily the same as the one in the dataset. For that matter, we do not have a guarantee that our labels refer to different species of iris. It is possible that the 

# 4. Since the Iris dataset does have class labels, we will evaluate the quality of our clustering by comparing the clusters to the actual species.

To do this, we must define a **cluster-label mapping**. For each cluster $c \in \{0, 1, 2\}$:
- Look at all the samples that your algorithm assigned to cluster $c$.
- Among those samples, identify the most common true label (majority vote).
- Map cluster $c$ to that label.

Using this mapping, define predicted labels $\hat{y}_i$ for all samples. Finally, define the **clustering accuracy** as

$$
\text{Accuracy} = \frac{1}{150} \|\{i: \hat{y}_i = y_i\}\|
$$

where $y_i$ are the true Iris labels. Report your mapping, predicted labels, confusion matrix, and accuracy.



**Example: cluster-label mapping and confusion matrix.**

Suppose a clustering algorithm assigns each sample to one of three clusters $c \in \{0,1,2\}$, and we obtain the following assignments for five samples:

$$
\begin{array}{c|c}
\text{Sample } i & \text{Cluster assigned} \\ \hline
1 & 0 \\
2 & 1 \\
3 & 1 \\
4 & 2 \\
5 & 2
\end{array}
$$

Assume the true labels of these samples are:

$$
\begin{array}{c|c}
\text{Sample } i & \text{True label} \\ \hline
1 & 0 \\
2 & 1 \\
3 & 2 \\
4 & 2 \\
5 & 0
\end{array}
$$

To compare clusters with true labels, we must define a *cluster-label mapping*. For each cluster, we look at the true labels of the samples inside that cluster:

$$
\begin{array}{c|c}
\text{Cluster} & \text{True labels inside it} \\ \hline
0 & \{0\} \\
1 & \{1,2\} \\
2 & \{2,0\}
\end{array}
$$

We assign each cluster to the **majority** true label:

$$
\begin{align*}
\pi(0) = 0, && \pi(1) = 1, && \pi(2) = 2
\end{align*}
$$

Using this mapping, we convert each cluster index into a predicted label. For example, if a sample was assigned to cluster 2, its predicted label is $\pi(2) = 2$.

This gives the following predicted-vs-true table:

$$
\begin{array}{c|c|c}
\text{Sample } i & \text{True} & \text{Predicted} \\ \hline
1 & 0 & 0 \\
2 & 1 & 1 \\
3 & 2 & 1 \\
4 & 2 & 2 \\
5 & 0 & 2 
\end{array}
$$

Now we form a **confusion matrix** (rows = true labels, columns = predicted labels):

$$
\begin{array}{c|ccc}
  & \hat{0} & \hat{1} & \hat{2} \\ \hline
0 & 1 & 0 & 1 \\
1 & 0 & 1 & 0 \\
2 & 0 & 1 & 1
\end{array}
$$

Finally, the clustering accuracy is

$$
\text{Accuracy} = \frac{\text{number of correct predictions}}{\text{total samples}} = \frac{1 + 1 + 1}{5} = 0.6
$$

This example illustrates (i) how cluster IDs must be mapped to labels, (ii) how predicted labels are computed, and (iii) how the confusion matrix is formed.

In [29]:
# Find most common true label in each cluster
# groupby() --> value_counts() produces a list of labels sorted by frequency
# agg(lambda x: x.value_counts().index[0]) picks the most frequent label
def cluster_map(df, true_col, cluster_col, pred_col):
    cluster_mapping = df[true_col].groupby(df[cluster_col]).agg(lambda x: x.value_counts().index[0])
    df[pred_col] = df[cluster_col].map(cluster_mapping)
    return df

iris_normalized = cluster_map(iris_normalized, 'variety', 'cluster_index_competitive', 'variety_pred_competitive')
iris_normalized

,sepal.length,sepal.width,petal.length,petal.width,variety,cluster_index_competitive,variety_pred_competitive
0,-0.897674,1.015602,-1.335752,-1.311052,Setosa,1,Setosa
1,-1.139200,-0.131539,-1.335752,-1.311052,Setosa,1,Setosa
2,-1.380727,0.327318,-1.392399,-1.311052,Setosa,1,Setosa
3,-1.501490,0.097889,-1.279104,-1.311052,Setosa,1,Setosa
4,-1.018437,1.245030,-1.335752,-1.311052,Setosa,1,Setosa
...,...,...,...,...,...,...,...
145,1.034539,-0.131539,0.816859,1.443994,Virginica,2,Virginica
146,0.551486,-1.278680,0.703564,0.919223,Virginica,0,Versicolor
147,0.793012,-0.131539,0.816859,1.050416,Virginica,2,Virginica
148,0.430722,0.786174,0.930154,1.443994,Virginica,2,Virginica


In [38]:
# Create a confusion matrix from true and predicted labels
def compute_confusion_matrix(df, true_col, pred_col):
    return pd.crosstab(df[true_col], df[pred_col], rownames=['Actual'], colnames=['Predicted'])

# Produce confusion matrix
confusion_matrix_competitive = compute_confusion_matrix(iris_normalized, 'variety', 'variety_pred_competitive')
print("Confusion Matrix:")
print(confusion_matrix_competitive)

Confusion Matrix:
Predicted   Setosa  Versicolor  Virginica
Actual                                   
Setosa          50           0          0
Versicolor       0          42          8
Virginica        0          18         32


In [37]:
# Compute accuracy
def compute_accuracy(df, true_col, pred_col):
    return np.sum(df[true_col] == df[pred_col]) / len(df)

accuracy_competitive = compute_accuracy(iris_normalized, 'variety', 'variety_pred_competitive')

print(f"Competitive Clustering Accuracy: {accuracy_competitive:.2f}")

Competitive Clustering Accuracy: 0.83


## 5. Now run the standard $k$-means algorithm with $k=3$ on the same standardized data $X$.

Again, compute cluster indices, define a cluster-label mapping by majority vote, and compute the accuracy and confusion matrix.


In [32]:
def kmeans(k, data, n_iters):
    # Randomly initialize centroids by selecting k random samples from data
    np.random.seed(12345)
    centroids = data[np.random.choice(data.shape[0], k, replace=False)]
    
    for iteration in range(n_iters):
        # Assignment step
        cluster_indices = []
        for x in data:
            distances = np.linalg.norm(centroids - x, axis=1)
            winner_idx = np.argmin(distances)
            cluster_indices.append(winner_idx)
        
        cluster_indices = np.array(cluster_indices)
        
        # Update step
        new_centroids = np.array([data[cluster_indices == i].mean(axis=0) for i in range(k)])
        
        # Check for convergence (if centroids do not change)
        if np.all(centroids == new_centroids):
            break
        
        centroids = new_centroids
    
    return centroids, cluster_indices

centroids, cluster_indices = kmeans(k=3, data=X, n_iters=100)

iris_normalized['cluster_index_kmeans'] = cluster_indices
iris_normalized = cluster_map(iris_normalized, 'variety', 'cluster_index_kmeans', 'variety_pred_kmeans')
iris_normalized

,sepal.length,sepal.width,petal.length,petal.width,variety,cluster_index_competitive,variety_pred_competitive,cluster_index_kmeans,variety_pred_kmeans
0,-0.897674,1.015602,-1.335752,-1.311052,Setosa,1,Setosa,1,Setosa
1,-1.139200,-0.131539,-1.335752,-1.311052,Setosa,1,Setosa,1,Setosa
2,-1.380727,0.327318,-1.392399,-1.311052,Setosa,1,Setosa,1,Setosa
3,-1.501490,0.097889,-1.279104,-1.311052,Setosa,1,Setosa,1,Setosa
4,-1.018437,1.245030,-1.335752,-1.311052,Setosa,1,Setosa,1,Setosa
...,...,...,...,...,...,...,...,...,...
145,1.034539,-0.131539,0.816859,1.443994,Virginica,2,Virginica,2,Virginica
146,0.551486,-1.278680,0.703564,0.919223,Virginica,0,Versicolor,0,Versicolor
147,0.793012,-0.131539,0.816859,1.050416,Virginica,2,Virginica,2,Virginica
148,0.430722,0.786174,0.930154,1.443994,Virginica,2,Virginica,2,Virginica


In [33]:
confusion_matrix_kmeans = compute_confusion_matrix(iris_normalized, 'variety', 'variety_pred_kmeans')

print("Confusion Matrix for K-Means:")
print(confusion_matrix_kmeans)

Confusion Matrix for K-Means:
Predicted   Setosa  Versicolor  Virginica
Actual                                   
Setosa          50           0          0
Versicolor       0          38         12
Virginica        0          11         39


In [40]:
accuracy_kmeans = compute_accuracy(iris_normalized, 'variety', 'variety_pred_kmeans')

print(f"K-Means Clustering Accuracy: {accuracy_kmeans:.2f}")

K-Means Clustering Accuracy: 0.85


## 6. Compare the two clustering methods. Your report should discuss:
- The differences in the learned clusters.
- Which method achieved a higher accuracy, and why this may be the case.
- Any sensitivity of your results to initialization or learning rate.
- A 2D scatter plot (using principle component analysis for dimensionality reduction - independent learning if you do not know it!) showing the final clusters for both methods.

Include screenshots or figures of your cluster plots and confusion matrices. As usual, upload your code to Box with the filename `06-IDNumber-LastName.py`

"Just figure out PCA as independent learning" in the *final instruction* of the assignment is evil lol
